In [ ]:
index2move = {}
move2index = {}

index = 0
for sr in range(10):  # 起始行
    for sc in range(9):  # 起始列
        for er in range(10):  # 目标行
            for ec in range(9):  # 目标列
                move = f"{sr}{sc}{er}{ec}"
                index2move[move] = index
                move2index[index] = move
                index += 1


In [ ]:
print(index2move['6050'])

4905


In [17]:
def generate_legal_moves():
    """
    Generate all possible legal moves for the chess game, including all legal moves
    for all chess pieces and fixed moves for specific pieces like advisors and bishops.
    """
    index2move = {}
    move2index = {}

    # Define row and column indices
    rows = range(10)  # 0 to 9
    cols = range(9)   # 0 to 8

    # Predefined legal moves for advisors and bishops
    advisor_moves = [
        (3, 0, 4, 1), (4, 1, 3, 0), (5, 0, 4, 1), (4, 1, 5, 0),
        (3, 2, 4, 1), (4, 1, 3, 2), (5, 2, 4, 1), (4, 1, 5, 2),
        (9, 3, 8, 4), (8, 4, 9, 3), (9, 5, 8, 4), (8, 4, 9, 5),
        (7, 3, 8, 4), (8, 4, 7, 3), (7, 5, 8, 4), (8, 4, 7, 5)
    ]
    bishop_moves = [
        (2, 0, 0, 2), (0, 2, 2, 0), (2, 4, 4, 2), (4, 2, 2, 4),
        (0, 2, 2, 4), (2, 4, 0, 2), (4, 2, 6, 0), (6, 0, 4, 2),
        (4, 6, 6, 4), (6, 4, 4, 6), (6, 8, 4, 6), (4, 6, 6, 8),
        (7, 0, 5, 2), (5, 2, 7, 0), (7, 8, 5, 6), (5, 6, 7, 8)
    ]

    # Helper function to convert (start_row, start_col, end_row, end_col) to action string
    def move_to_action(start, end):
        return f"{start[0]}{start[1]}{end[0]}{end[1]}"

    # Generate legal moves for general pieces
    move_idx = 0
    for start_row in rows:
        for start_col in cols:
            # Possible destinations
            horizontal_moves = [(start_row, c) for c in cols]  # Same row
            vertical_moves = [(r, start_col) for r in rows]    # Same column
            knight_moves = [
                (start_row + dr, start_col + dc)
                for dr, dc in [(-2, -1), (-1, -2), (-2, 1), (1, -2), (2, -1), (-1, 2), (2, 1), (1, 2)]
                if 0 <= start_row + dr < 10 and 0 <= start_col + dc < 9
            ]

            all_moves = set(horizontal_moves + vertical_moves + knight_moves)
            for end_pos in all_moves:
                if (start_row, start_col) != end_pos:  # Avoid no-op moves
                    action = move_to_action((start_row, start_col), end_pos)
                    index2move[move_idx] = action
                    move2index[action] = move_idx
                    move_idx += 1

    # Add predefined moves for advisors
    for move in advisor_moves:
        action = move_to_action((move[0], move[1]), (move[2], move[3]))
        index2move[move_idx] = action
        move2index[action] = move_idx
        move_idx += 1

    # Add predefined moves for bishops
    for move in bishop_moves:
        action = move_to_action((move[0], move[1]), (move[2], move[3]))
        index2move[move_idx] = action
        move2index[action] = move_idx
        move_idx += 1

    return index2move, move2index


index2move, move2index = generate_legal_moves()


In [22]:
import numpy as np
current_state = np.zeros((10, 9), dtype=int)
current_state[0] = [-1, -2, -3, -4, -5, -4, -3, -2, -1]
current_state[9] = [1, 2, 3, 4, 5, 4, 3, 2, 1]
current_state[2][1] = current_state[2][7] = -6
current_state[7][1] = current_state[7][7] = 6
current_state[3][0::2] = -7
current_state[6][0::2] = 7

In [23]:
print(current_state)

[[-1 -2 -3 -4 -5 -4 -3 -2 -1]
 [ 0  0  0  0  0  0  0  0  0]
 [ 0 -6  0  0  0  0  0 -6  0]
 [-7  0 -7  0 -7  0 -7  0 -7]
 [ 0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0]
 [ 7  0  7  0  7  0  7  0  7]
 [ 0  6  0  0  0  0  0  6  0]
 [ 0  0  0  0  0  0  0  0  0]
 [ 1  2  3  4  5  4  3  2  1]]


In [27]:
def is_face_to_face(state):
        """
        Check whether the generals are face to face
        """
        k_x, k_y = None, None
        K_x, K_y = None, None

        for y in range(10):
            for x in range(9):
                piece = state[y][x]
                if piece == 5:  # 
                    K_x, K_y = x, y
                elif piece == -5:  # 
                    k_x, k_y = x, y
        print(k_x,k_y, K_x, K_y)
        if k_x == K_x:  # in the same column
            for i in range(min(k_y, K_y) + 1, max(k_y, K_y)):
                if state[i][k_x] != 0:
                    return False
            return True
        return False

In [28]:
is_face_to_face(current_state)

4 0 4 9


False

In [2]:
from torchsummary import summary
from Net import PolicyValueNet

model = PolicyValueNet(model_file="models\current_model.pth")
summary(model.net, input_size=(9, 10, 9))  # 输入的形状与训练时一致


<>:4: SyntaxWarning: invalid escape sequence '\c'
<>:4: SyntaxWarning: invalid escape sequence '\c'
C:\Users\Altman\AppData\Local\Temp\ipykernel_172876\2870666972.py:4: SyntaxWarning: invalid escape sequence '\c'
  model = PolicyValueNet(model_file="models\current_model.pth")


Model Loaded Successfully: models\current_model.pth
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 256, 10, 9]          20,992
       BatchNorm2d-2           [-1, 256, 10, 9]             512
              ReLU-3           [-1, 256, 10, 9]               0
            Conv2d-4           [-1, 256, 10, 9]         590,080
       BatchNorm2d-5           [-1, 256, 10, 9]             512
              ReLU-6           [-1, 256, 10, 9]               0
            Conv2d-7           [-1, 256, 10, 9]         590,080
       BatchNorm2d-8           [-1, 256, 10, 9]             512
              ReLU-9           [-1, 256, 10, 9]               0
         ResBlock-10           [-1, 256, 10, 9]               0
           Conv2d-11           [-1, 256, 10, 9]         590,080
      BatchNorm2d-12           [-1, 256, 10, 9]             512
             ReLU-13           [-1, 256, 10, 9]    

In [ ]:
import numpy as np
import torch
from Net import PolicyValueNet
from game import GameState
from config import CONFIG

def verify_model(model_path, board_state):
    policy_value_net = PolicyValueNet(model_file=model_path)
    print("Model Loaded Successfully.")
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    policy_value_net.net.to(device) 
    

    game = GameState()
    game.current_state = np.array(board_state)
    training_state = game.get_training_state()

    training_state = np.expand_dims(training_state, axis=0)
    training_state = torch.tensor(training_state, dtype=torch.float32).to(device)
    
    with torch.no_grad():
        log_act_probs, value = policy_value_net.net(training_state)
        act_probs = torch.exp(log_act_probs).cpu().numpy().flatten()
        value = value.item()

    print("Predicted Action Probabilities:")
    print(act_probs)

    print("\nPredicted State Value (Win Probability):")
    print(value)

    top_moves = np.argsort(act_probs)[-5:][::-1] 
    print("\nTop 5 Moves (Action IDs and Probabilities):")
    for move in top_moves:
        print(f"Action ID: {move}, Probability: {act_probs[move]}")

example_board_state = [
    [ 0,  0,  0,  -5,  0,  0,  0,  0, 0],
    [ 0,  0,  0,  0,  0,  0,  0,  0,  0],
    [ 0,  0,  0,  0,  1,  0,  0,  0,  0],
     [0,  0,  0,  0,  0,  0, 0,  0, 0],
    [ 0,  0,  0,  0,  0,  1,  0,  0,  0],
    [ 0,  0,  0,  0,  0,  0,  0,  0,  0],
    [ 7,  0,  7,  0,  7,  0,  7,  0,  7],
    [ 0,  0,  0,  0,  0,  0,  0,  0,  0],
    [ 0,  0,  0,  0,  0,  0,  0,  0,  0],
    [ 0,  2,  3,  0,  5,  4,  3,  2,  0]
]

verify_model(CONFIG["model_file"], example_board_state)


Model Loaded Successfully: ./models/current_model.pth
Model Loaded Successfully.
Predicted Action Probabilities:
[1.7097061e-04 1.6836829e-03 8.6662371e-04 ... 1.8955900e-04 9.9657918e-05
 1.7092627e-04]

Predicted State Value (Win Probability):
-0.9979957938194275

Top 5 Moves (Action IDs and Probabilities):
Action ID: 1328, Probability: 0.20368847250938416
Action ID: 55, Probability: 0.03019220009446144
Action ID: 2030, Probability: 0.027153007686138153
Action ID: 1893, Probability: 0.02488146349787712
Action ID: 2048, Probability: 0.017499295994639397


In [6]:
from game import index2move, move2index
print(index2move[55])
print(index2move[1328])

0203
6454
